In [ ]:
import os
import sys
import uuid
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from src.services.main import TextDetection
plt.rcParams['figure.dpi'] = 300

In [ ]:
nb_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

In [ ]:
import config
import src.utilities.app_context as app_context
app_context.init()

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH= nb_dir + '/utilities/craft_pytorch/model/craft_mlt_25k.pth'
config.CRAFT_REFINE_MODEL_PATH = nb_dir + '/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth'

In [ ]:
def draw_region(page_path,corrds,color= (255,0,0),thickness=5, save=False):
    
    if type(page_path) == str :  
        image = cv2.imread(page_path) 
    else :
        image = page_path
    for region in corrds :
            ground = region['boundingBox']['vertices']
            #start_point  = (ground[0]['x'],ground[0]['y'])
            #end_point = (ground[2]['x'],  ground[2]['y'])
            #cv2.rectangle(image, start_point, end_point, color,thickness)
            pts = []
            for pt in ground:
                pts.append([pt['x'] ,pt['y']])
            
            cv2.polylines(image, [np.array(pts)],True, color, thickness)
    plt.imshow(image)
    if save:
        cv2.imwrite(str(uuid.uuid1()) + '.png' , image)
        
    return image


In [ ]:
base_dir = '/home/siddanth/Desktop/craft/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/word-detector/craft/upload'

filename    = 'crafted.pdf'
file_format = 'PDF'
language    =  'en'

app_context.application_context = {
    "input": {
        "inputs": [
            {
                "file": {
                    "identifier": "string",
                    "name": "crafted.pdf",
                    "type": "PDF"
                },
                "config": {
                    "OCR": {
                        "option": "HIGH_ACCURACY",
                        "language": "en",
                        "top_correction": "False",
                        "craft_word": "False",
                        "craft_line": "False",
                        "line_layout": "True",
                        "mask_image": "False"
                    }
                }
            }
        ]
    }
}


In [ ]:
resp = TextDetection(app_context,base_dir)
filepath = resp['rsp']['outputs'][0]['page_info'][0]
save_dir = '/home/siddanth/Desktop/craft/'

In [ ]:
def draw_box(resp,filepath,save_dir,color="green", save=False):
    image  = Image.open(filepath)
    draw   = ImageDraw.Draw(image)
    for i in resp['rsp']['outputs'][0]['pages'][0]['lines']:
        draw.rectangle(((i['boundingBox']['vertices'][0]['x'], i['boundingBox']['vertices'][0]['y']), (i['boundingBox']['vertices'][2]['x'],i['boundingBox']['vertices'][2]['y'])), outline=color,width=3)

    save_filepath = os.path.join(save_dir, "bbox_"+os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image
#draw_box(resp,filepath, save_dir, color='green', save=True)

In [ ]:
file_index = 0
page_index =0
to_check  = 'words'
regions    = resp['rsp']['outputs'][file_index]['pages'][page_index][to_check]
page_path = resp['rsp']['outputs'][file_index]['page_info'][page_index]

#draw_region(page_path,regions)
